## <span style="font-family: Latin Modern Roman; font-size: 25px;"> Fact-Checking with Google Fact Check Tools API </span>

In [1]:
import os
import requests
import pandas as pd

current_dir = os.getcwd()  # Current working directory for the notebook
api_key_path = os.path.join(current_dir, "../api/api_key.txt")  # Adjust relative path
with open(api_key_path, "r") as file:
    API_KEY = file.read().strip()

# Languages supported
LANGUAGE_MAP = {
    "en": "English",
    "es": "Spanish",
}

def search_fact_check_claims(api_key, query, language_code="en"):
    url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

    params = {
        "key": api_key,
        "query": query,
        "languageCode": language_code,
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()

        data = response.json()
        claims = data.get("claims", [])

        results = []
        for claim in claims:
            text = claim.get("text", "No text available")
            claimant = claim.get("claimant", "Unknown")
            review = claim.get("claimReview", [])[0] if claim.get("claimReview") else None
            
            if review:
                publisher = review.get("publisher", {}).get("name", "Unknown publisher")
                rating = review.get("textualRating", "No rating")
                url = review.get("url", "No URL")
                review_date = review.get("reviewDate", "No date")

                results.append({
                    "Claim": text,
                    "Claimant": claimant,
                    "Publisher": publisher,
                    "Rating": rating,
                    "Date": review_date,
                    "URL": url
                })

        if results:
            return pd.DataFrame(results)
        else:
            language_name = LANGUAGE_MAP.get(language_code, language_code)
            print(f"No claims matching '{query}' were found in {language_name}.\n\nSuggestions:\n- Make sure all keywords are spelled correctly.\n- Try different keywords.\n- Try more general keywords.\n- Try fewer keywords.")
            return pd.DataFrame()

    except requests.exceptions.RequestException as e:
        print(f"An error ocurred: {e}")
        return pd.DataFrame()

In [2]:
claim_to_check = "The earth is flat."
results_df = search_fact_check_claims(API_KEY, claim_to_check)
results_df

,Claim,Claimant,Publisher,Rating,Date,URL
0,Flights from New Zealand to Argentina connect ...,Viral social media post,Science Feedback,Inaccurate,2025-02-26T00:00:00Z,https://science.feedback.org/review/flight-pat...
1,The Earth is flat.,instagram user,Full Fact,We have abundant evidence going back thousands...,2023-03-03T00:00:00Z,https://fullfact.org/online/earth-is-spherical...
2,Satellites are fake.,instagram user,Full Fact,There are many man-made satellites currently i...,2023-03-03T00:00:00Z,https://fullfact.org/online/earth-is-spherical...
3,The Earth is flat because cities cannot be ups...,Social media,USA Today,False,2022-12-13T21:25:17Z,https://www.usatoday.com/story/news/factcheck/...
4,The Earth is flat based on various observation...,Social media users,Science Feedback,Incorrect: It has been well established by mea...,2024-12-05T00:00:00Z,https://science.feedback.org/review/how-we-kno...
5,NASA admits the Earth is flat and non-rotating,Social media,USA Today,False,2023-01-10T21:55:58Z,https://www.usatoday.com/story/news/factcheck/...
6,"Pictures show the Earth is flat, and sea level...",Social media,USA Today,False,2023-08-07T21:38:36Z,https://www.usatoday.com/story/news/factcheck/...
7,Radar technology wouldn’t work if the Earth wa...,Social media,USA Today,False,2023-05-08T23:13:06Z,https://www.usatoday.com/story/news/factcheck/...
8,"Laser tests show bodies of water are level, wh...",Social media,USA Today,False,2022-12-07T21:53:01Z,https://www.usatoday.com/story/news/factcheck/...
9,Video shows Elon Musk admitting he is a flat e...,Social media,USA Today,Altered,2023-04-28T13:59:26Z,https://www.usatoday.com/story/news/factcheck/...


In [6]:
claim_to_check = "Gun manufacturers are the only industry in the country that have immunity from lawsuits."
results_df = search_fact_check_claims(API_KEY, claim_to_check)
results_df

,Claim,Claimant,Publisher,Rating,URL
0,Gun manufacturers are “the only industry in th...,Joe Biden,PolitiFact,False,https://www.politifact.com/factchecks/2022/jun...


In [7]:
claim_to_check = "The EU will ban 'climate-neutral' claims by 2026."
results_df = search_fact_check_claims(API_KEY, claim_to_check)
results_df

No claims matching 'The EU will ban 'climate-neutral' claims by 2026.' were found in English.

Suggestions:
- Make sure all keywords are spelled correctly.
- Try different keywords.
- Try more general keywords.
- Try fewer keywords.


""


In [8]:
claim_to_check = "Ursula von der Leyen has said: 'The European Union is forced to buy Russian oil so Putin does not earn more somewhere else.'"
results_df = search_fact_check_claims(API_KEY, claim_to_check)
results_df

,Claim,Claimant,Publisher,Rating,URL
0,"Ursula von der Leyen has said: ""The European U...",Unknown,Logically Facts,Misleading,https://www.logicallyfacts.com/en/fact-check/o...


In [4]:
claim_to_check = "Christine Lagarde ha dicho que el BCE va a eliminar el dinero en efectivo por el cambio climático"
results_df = search_fact_check_claims(API_KEY, claim_to_check, language_code="es")
results_df

,Claim,Claimant,Publisher,Rating,URL
0,La presidenta del BCE asegura en un vídeo que ...,Unknown,EFE Verifica,Es falso,https://verifica.efe.com/la-directora-del-bce-...
1,Christine Lagarde dice que hay que eliminar el...,Unknown,ColombiaCheck,Falso,https://colombiacheck.com/chequeos/directora-d...
2,"Christine Lagarde, presidenta del Banco Centra...",Unknown,Fast Check,Falso,https://www.fastcheck.cl/2024/12/27/christine-...
